# Análisis General de Diferencia de Diferencias

Este notebook analiza las variables DifDif calculadas en el notebook 48.

## Análisis realizados:

1. **Estadísticas descriptivas**: Media, n, Desvío Estándar
2. **Tests de significancia**: Test t de una muestra (H₀: DifDif = 0)
3. **Tablas Excel**: Resultados formateados y exportados

## Interpretación:

- Si DifDif es **significativamente diferente de 0**, significa que hubo un cambio real en la asimetría Izq-Der entre Generales y Ballotage.

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

## 1. Cargar Datos

In [ ]:
# Cargar df_Elecciones
Ruta_Datos = os.path.join(os.getcwd(), '..', 'Data', 'Procesados', 'df_Elecciones.xlsx')
df_Elecciones = pd.read_excel(Ruta_Datos)

print(f"✓ df_Elecciones cargado exitosamente")
print(f"  Dimensiones: {df_Elecciones.shape}")
print(f"  Columnas: {df_Elecciones.columns.tolist()[:5]}...")

## 2. Identificar Variables DifDif

In [ ]:
# Identificar columnas DifDif
columnas_difdif_co = [col for col in df_Elecciones.columns if col.startswith('DifDif_CO')]
columnas_difdif_ct = [col for col in df_Elecciones.columns if col.startswith('DifDif_CT')]

print(f"Variables DifDif_CO: {len(columnas_difdif_co)}")
print(f"Variables DifDif_CT: {len(columnas_difdif_ct)}")
print(f"\nEjemplos DifDif_CO: {columnas_difdif_co[:3]}")
print(f"Ejemplos DifDif_CT: {columnas_difdif_ct[:3]}")

## 3. Función para Calcular Estadísticas y Significancia

In [ ]:
def Calcular_Estadisticas_DifDif(df, columnas_difdif):
    """
    Calcula estadísticas descriptivas y test t para variables DifDif.
    
    Test de hipótesis:
    H₀: DifDif = 0 (no hay cambio en la asimetría)
    H₁: DifDif ≠ 0 (hay cambio en la asimetría)
    """
    
    resultados = []
    
    for columna in columnas_difdif:
        # Extraer datos válidos (sin NaN)
        datos = df[columna].dropna()
        
        if len(datos) > 0:
            # Estadísticas descriptivas
            media = datos.mean()
            n = len(datos)
            de = datos.std()
            ee = datos.sem()
            
            # Test t de una muestra (contra 0)
            if n > 1:
                t_stat, p_valor = stats.ttest_1samp(datos, 0)
            else:
                t_stat = np.nan
                p_valor = np.nan
            
            # Determinar significancia
            if not np.isnan(p_valor):
                if p_valor < 0.001:
                    sig = '***'
                elif p_valor < 0.01:
                    sig = '**'
                elif p_valor < 0.05:
                    sig = '*'
                else:
                    sig = 'ns'
            else:
                sig = '—'
            
            resultados.append({
                'Variable': columna,
                'Media': media,
                'n': n,
                'DE': de,
                'EE': ee,
                't': t_stat,
                'p_valor': p_valor,
                'Sig': sig
            })
        else:
            resultados.append({
                'Variable': columna,
                'Media': np.nan,
                'n': 0,
                'DE': np.nan,
                'EE': np.nan,
                't': np.nan,
                'p_valor': np.nan,
                'Sig': '—'
            })
    
    return pd.DataFrame(resultados)

## 4. Calcular Estadísticas para DifDif_CO

In [ ]:
print("Calculando estadísticas para DifDif_CO...\n")

df_Stats_CO = Calcular_Estadisticas_DifDif(df_Elecciones, columnas_difdif_co)

print(f"✓ Estadísticas calculadas para {len(df_Stats_CO)} variables DifDif_CO")
print(f"\nPrimeras 5 variables:\n")
print(df_Stats_CO.head())

## 5. Calcular Estadísticas para DifDif_CT

In [ ]:
print("Calculando estadísticas para DifDif_CT...\n")

df_Stats_CT = Calcular_Estadisticas_DifDif(df_Elecciones, columnas_difdif_ct)

print(f"✓ Estadísticas calculadas para {len(df_Stats_CT)} variables DifDif_CT")
print(f"\nPrimeras 5 variables:\n")
print(df_Stats_CT.head())

## 6. Resumen de Significancia

In [ ]:
print("="*70)
print("RESUMEN DE SIGNIFICANCIA")
print("="*70)

print("\nDifDif_CO:")
print(df_Stats_CO['Sig'].value_counts().sort_index())

print("\nDifDif_CT:")
print(df_Stats_CT['Sig'].value_counts().sort_index())

print("\n" + "="*70)

## 7. Funciones Auxiliares para Excel

In [ ]:
def hex_to_rgb(hex_color):
    """Convierte color hexadecimal a formato RGB para openpyxl."""
    hex_color = hex_color.lstrip('#')
    return hex_color.upper()


def aplicar_borde_completo(cell):
    """Aplica bordes a todas las celdas."""
    thin_border = Border(
        left=Side(style='thin', color='000000'),
        right=Side(style='thin', color='000000'),
        top=Side(style='thin', color='000000'),
        bottom=Side(style='thin', color='000000')
    )
    cell.border = thin_border


def obtener_color_significancia(texto_sig):
    """Retorna el color de fondo según nivel de significancia."""
    if texto_sig == '***':
        return '90EE90'  # Verde oscuro
    elif texto_sig == '**':
        return 'B8F4B8'  # Verde medio
    elif texto_sig == '*':
        return 'D4F4D4'  # Verde claro
    elif texto_sig == 'ns':
        return 'FFE4E1'  # Rosa claro
    else:
        return 'FFFFFF'  # Blanco


def obtener_color_texto_significancia(texto_sig):
    """Retorna el color de texto según nivel de significancia."""
    if texto_sig == '***':
        return '006400'  # Verde oscuro
    elif texto_sig == '**':
        return '228B22'  # Verde medio
    elif texto_sig == '*':
        return '000000'  # Negro
    elif texto_sig == 'ns':
        return '8B0000'  # Rojo oscuro
    else:
        return '000000'  # Negro

## 8. Función para Crear Tabla Excel General

In [ ]:
def Crear_Tabla_Excel_DifDif_General(
    df_stats,
    tipo,
    nombre_archivo,
    carpeta_destino='Tablas_Excel_DifDif'
):
    """
    Crea una tabla Excel con estadísticas generales de DifDif.
    
    Parámetros:
    - df_stats: DataFrame con estadísticas calculadas
    - tipo: 'CO' o 'CT'
    - nombre_archivo: Nombre del archivo de salida
    - carpeta_destino: Carpeta donde guardar
    """
    
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    # Crear workbook
    wb = Workbook()
    ws = wb.active
    ws.title = f'DifDif_{tipo}_General'
    
    fila = 1
    
    # TÍTULO
    ws.merge_cells(start_row=fila, start_column=1, end_row=fila, end_column=7)
    cell_titulo = ws.cell(fila, 1)
    cell_titulo.value = f'Análisis General - Diferencia de Diferencias ({tipo})'
    cell_titulo.font = Font(bold=True, size=14, color='FFFFFF')
    cell_titulo.fill = PatternFill(start_color='2C5282', end_color='2C5282', fill_type='solid')
    cell_titulo.alignment = Alignment(horizontal='center', vertical='center')
    
    fila += 2
    
    # ENCABEZADO
    headers = ['Ítem', 'Media', 'n', 'DE', 't', 'p-valor', 'Sig']
    for col_idx, header in enumerate(headers, start=1):
        cell = ws.cell(fila, col_idx)
        cell.value = header
        cell.font = Font(bold=True, color='FFFFFF', size=11)
        cell.fill = PatternFill(start_color='5B9BD5', end_color='5B9BD5', fill_type='solid')
        cell.alignment = Alignment(horizontal='center', vertical='center')
    
    fila += 1
    
    # DATOS
    for idx, row in df_stats.iterrows():
        # Extraer número de ítem de la variable
        variable = row['Variable']
        # Formato: DifDif_CO_Item_5 o DifDif_CT_Item_5
        num_item = variable.split('_')[-1]
        
        # Ítem
        cell = ws.cell(fila, 1)
        cell.value = f'Ítem {num_item}'
        cell.alignment = Alignment(horizontal='center', vertical='center')
        cell.fill = PatternFill(start_color='E7E6E6', end_color='E7E6E6', fill_type='solid')
        cell.font = Font(bold=True)
        
        # Media
        cell = ws.cell(fila, 2)
        if not np.isnan(row['Media']):
            cell.value = f"{row['Media']:.3f}"
        else:
            cell.value = '—'
        cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # n
        cell = ws.cell(fila, 3)
        cell.value = int(row['n']) if row['n'] > 0 else '—'
        cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # DE
        cell = ws.cell(fila, 4)
        if not np.isnan(row['DE']):
            cell.value = f"{row['DE']:.3f}"
        else:
            cell.value = '—'
        cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # t
        cell = ws.cell(fila, 5)
        if not np.isnan(row['t']):
            cell.value = f"{row['t']:.3f}"
        else:
            cell.value = '—'
        cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # p-valor
        cell = ws.cell(fila, 6)
        if not np.isnan(row['p_valor']):
            if row['p_valor'] < 0.001:
                cell.value = '<0.001'
            else:
                cell.value = f"{row['p_valor']:.4f}"
        else:
            cell.value = '—'
        cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # Significancia
        cell = ws.cell(fila, 7)
        cell.value = row['Sig']
        cell.alignment = Alignment(horizontal='center', vertical='center')
        color_fondo = obtener_color_significancia(row['Sig'])
        color_texto = obtener_color_texto_significancia(row['Sig'])
        cell.fill = PatternFill(start_color=color_fondo, end_color=color_fondo, fill_type='solid')
        cell.font = Font(bold=True, color=color_texto)
        
        fila += 1
    
    # Aplicar bordes
    for row in ws.iter_rows(min_row=1, max_row=fila-1, min_col=1, max_col=7):
        for cell in row:
            aplicar_borde_completo(cell)
    
    # Ajustar anchos de columna
    ws.column_dimensions['A'].width = 12
    ws.column_dimensions['B'].width = 12
    ws.column_dimensions['C'].width = 8
    ws.column_dimensions['D'].width = 12
    ws.column_dimensions['E'].width = 12
    ws.column_dimensions['F'].width = 12
    ws.column_dimensions['G'].width = 8
    
    # Guardar
    ruta_completa = os.path.join(carpeta_destino, nombre_archivo)
    wb.save(ruta_completa)
    
    print(f"✅ Tabla Excel guardada en: {ruta_completa}")
    
    return ruta_completa

## 9. Generar Tablas Excel

In [ ]:
# Tabla para DifDif_CO
print("Generando tabla Excel para DifDif_CO...\n")
Crear_Tabla_Excel_DifDif_General(
    df_stats=df_Stats_CO,
    tipo='CO',
    nombre_archivo='Tabla_DifDif_CO_General.xlsx'
)

# Tabla para DifDif_CT
print("\nGenerando tabla Excel para DifDif_CT...\n")
Crear_Tabla_Excel_DifDif_General(
    df_stats=df_Stats_CT,
    tipo='CT',
    nombre_archivo='Tabla_DifDif_CT_General.xlsx'
)

print("\n✅ Todas las tablas generadas exitosamente")

## 10. Resumen Final

In [ ]:
print("="*70)
print("RESUMEN DEL ANÁLISIS GENERAL DE DIFDIF")
print("="*70)

print(f"\n📊 Variables analizadas:")
print(f"   - DifDif_CO: {len(df_Stats_CO)} variables")
print(f"   - DifDif_CT: {len(df_Stats_CT)} variables")

print(f"\n📈 Resultados significativos (p < 0.05):")
sig_co = df_Stats_CO[df_Stats_CO['Sig'].isin(['*', '**', '***'])]
sig_ct = df_Stats_CT[df_Stats_CT['Sig'].isin(['*', '**', '***'])]
print(f"   - DifDif_CO: {len(sig_co)} de {len(df_Stats_CO)} ({len(sig_co)/len(df_Stats_CO)*100:.1f}%)")
print(f"   - DifDif_CT: {len(sig_ct)} de {len(df_Stats_CT)} ({len(sig_ct)/len(df_Stats_CT)*100:.1f}%)")

print(f"\n📁 Archivos Excel generados:")
print(f"   - Tabla_DifDif_CO_General.xlsx")
print(f"   - Tabla_DifDif_CT_General.xlsx")

print(f"\n🎯 Interpretación:")
print(f"   - Test utilizado: t de una muestra (H₀: DifDif = 0)")
print(f"   - DifDif significativo → Cambió la asimetría Izq-Der entre elecciones")
print(f"   - Media positiva → Mayor asimetría hacia Izq en Ballotage")
print(f"   - Media negativa → Mayor asimetría hacia Der en Ballotage")

print("\n" + "="*70)